In [0]:
#####
# Description: Main train Script for SVM model
# Author: Tirupal Rao Ravilla
# Date: 05/2019
#####
import numpy as np
import pandas as pd 
import tensorflow as tf

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, CuDNNLSTM, Bidirectional
from keras.layers.embeddings import Embedding
# from tensorflow.keras.backend import set_session
import keras.backend as K
import nltk
import string
import re

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


['sample_submission.csv', 'test.csv', 'train.csv']


In [0]:
df = pd.read_csv('../input/train.csv')

In [0]:
def binary(y):
  if y>=0.5:
      return 1
  else:
      return 0
def preprocess(text):
  text = re.sub(r'\W+',' ',text.lower())
  return text

In [0]:
df.shape

(1804874, 45)

In [0]:
x = df['comment_text'].apply(preprocess)
y = df['target'].apply(binary)

In [0]:
print(x.head())
print(y.head())
print(len(x),len(y))

0    this is so cool it s like would you want your ...
1    thank you this would make my life a lot less a...
2    this is such an urgent design problem kudos to...
3    is this something i ll be able to install on m...
4                 haha you guys are a bunch of losers 
Name: comment_text, dtype: object
0    0
1    0
2    0
3    0
4    1
Name: target, dtype: int64
1804874 1804874


In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)

In [0]:
vocabulary_size = len(tokenizer.word_index)
print(vocabulary_size)

309011


In [0]:
sequences = tokenizer.texts_to_sequences(x)
tr_data = pad_sequences(sequences, maxlen=145)

print(len(tr_data))

1804874


In [0]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(tr_data,y,test_size = 0.25,random_state = 42)

In [0]:
K.clear_session()
model = Sequential()
model.add(Embedding(vocabulary_size, 145, input_length=145))
model.add(LSTM(145, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.fit(tr_data, y, validation_split=0.25, epochs=3,batch_size=128)
model.fit(x_train, y_train, validation_data=(x_test,y_test), epochs=3,batch_size=5000)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 1353655 samples, validate on 451219 samples
Epoch 1/3
1353655/1353655 [==============================] - 181s 134us/step - loss: 0.2106 - acc: 0.9329 - val_loss: 0.1408 - val_acc: 0.9472
Epoch 2/3
1353655/1353655 [==============================] - 176s 130us/step - loss: 0.1319 - acc: 0.9493 - val_loss: 0.1332 - val_acc: 0.9490
Epoch 3/3
1353655/1353655 [==============================] - 176s 130us/step - loss: 0.1224 - acc: 0.9516 - val_loss: 0.1303 - val_acc: 0.9496


In [0]:
test= pd.read_csv('../input/test.csv')
xt = test['comment_text'].apply(preprocess)
sequences = tokenizer.texts_to_sequences(xt)
ts_data = pad_sequences(sequences, maxlen=145)



In [0]:
yts= model.predict(ts_data)

In [0]:
print(yts[:10])

[[1.3470352e-03]
 [6.7734718e-04]
 [6.5582991e-04]
 [7.8621507e-04]
 [9.1255534e-01]
 [1.9651651e-04]
 [1.4633238e-03]
 [6.2867999e-03]
 [5.6406915e-02]
 [1.1658669e-03]]


In [0]:
submission = pd.DataFrame([test['id']]).T
submission['prediction'] = yts
print(submission.head())

        id  prediction
0  7000000    0.001347
1  7000001    0.000677
2  7000002    0.000656
3  7000003    0.000786
4  7000004    0.912555


In [0]:
submission.to_csv('submission.csv', index=False)